In [ ]:
import sys
sys.path.append('../')
import daq
import file
import globalVAR as Gvar
import visa
import numpy as np
import time
from random import randint
import PyCapture2 as pc2

%load_ext autoreload
%autoreload 1

In [ ]:
DAQ = daq.Daq(desc='Test Descritpion')
DAQ.connect_instr('TDS2024C', 1)
DAQ.connect_instr('KA3005P', 'ttyUSB0')

In [ ]:
DAQ.send_command(DAQ.command_queue['ttyUSB0'], 'turn_on')
DAQ.send_command(DAQ.command_queue['C046401'], 'acquire_off')
DAQ.send_command(DAQ.command_queue['C046401'], 'set_acquisition_stop', ("SEQuence",))
DAQ.send_command(DAQ.command_queue['C046401'], 'set_acquisition_mode', ("AVErage",))
DAQ.send_command(DAQ.command_queue['C046401'], 'set_average_num', (128,))

In [ ]:
v = np.linspace(5.0, 20, 16)

for i in range(len(v)):
    DAQ.send_command(DAQ.command_queue['ttyUSB0'], 'set_record_voltage', (v[i],))
    time.sleep(0.1)
    DAQ.send_command(DAQ.command_queue['C046401'], 'acquire_on')
    time.sleep(13)
    DAQ.send_command(DAQ.command_queue['C046401'], 'save_waveform')
    time.sleep(1)

In [ ]:
DAQ.send_command(DAQ.command_queue['ttyUSB0'], 'turn_off')

In [ ]:
DAQ.end_dataset()

In [ ]:
for key in DAQ.instr:
    for elem in DAQ.instr[key]:
        DAQ.disconnect_instr(key, elem)